In [3]:
import sqlite3
import datetime as datetime
conn = sqlite3.connect("/plos_data/testDB.db")

In [4]:
cur = conn.cursor()

In [6]:
import os
import itertools
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from operator import add
#path = 'C:/Users/Pragati Shah/subset/'
#path1='C:/Users/Pragati Shah/Downloads/sample_10000/sample_10000/'
path2='PLOS Journals/PLoS_Pathog'

In [7]:
#This function takes a directory path, parses all the xml files present in the directory, and returns the key attributes in
#the format of a dataframe.
import glob
def articleToDF(path):
    mainDF=pd.DataFrame(columns=["doi","pmid","journal","publisher_id","publisher_name","authors","editors","subjects","title","alt_title","date_pub","date_rec","abstract","body"])
    tempDF=pd.DataFrame()
    
    for fn in glob.iglob(path + '**/*.nxml', recursive=True):
        author_surnames=[]
        author_names=[]
        editor_surnames=[]
        editor_names=[]
        subjects=[]
        title=" "
        alt_title=" "
        authors=[]
        editors=[]
        abstract=" "
        body=" "
        journal=" "
        publisher_id=" "
        publisher_name=" "
        rday=0
        rmonth=0
        ryear=0
        pday=0
        pmonth=0
        pyear=0
        tree = ET.parse(fn)  
        root = tree.getroot() 
        for elem in tree.iter(tag='article-id'):
            a=str(elem.attrib)
            if (a.find('pmid')>-1):        
                pmid=elem.text
            if (a.find('doi')>-1):        
                doi=elem.text
        for elem in tree.iter(tag='contrib'):
            a=str(elem.attrib)
            if (a.find('author')>-1):
                for i in elem.iter(tag='surname'):
                    if i.text is not None:
                    #print (i.tag, i.attrib,i.text)
                        author_surnames.append(i.text)
                    else:
                        author_surnames.append("")
                for i in elem.iter(tag='given-names'):
                    #print (i.tag, i.attrib,i.text)
                    if i.text is not None:
                    #print (i.tag, i.attrib,i.text)
                        author_names.append(i.text)
                    else:
                        author_names.append("")
            spaces=' '* len(author_names)
            authors=list(map(add,list(map(add,author_names,spaces)),author_surnames))
            #authors=author_names
            if (a.find('editor')>-1):
                for i in elem.iter(tag='surname'):
                    if i.text is not None:
                    #print (i.tag, i.attrib,i.text)
                        editor_surnames.append(i.text)
                    else:
                        editor_surnames.append("")
                for i in elem.iter(tag='given-names'):
                    #print (i.tag, i.attrib,i.text)
                    if i.text is not None:
                    #print (i.tag, i.attrib,i.text)
                        editor_names.append(i.text)
                    else:
                        editor_names.append("")

            spaces=' '* len(editor_names)
            editors=list(map(add,list(map(add, editor_names,spaces)),editor_surnames))
            #editors=editor_names       
        for a in root.iter('article-meta'):
            for elem in a.iter('article-title'):
                title=elem.text
            for elem in a.iter('alt-title'):
                alt_title=elem.text
            for elem in a.iter('abstract'):    
                b=ET.tostring(elem, method='text', encoding='unicode')
                abstract=b
         
        for subject in root.iter('subject'):
            subjects.append(subject.text)
            
        for a in root.iter('pub-date'):
            for elem in a.iter('day'):
                pday=elem.text
            for elem in a.iter('month'):
                pmonth=elem.text
            for elem in a.iter('year'):
                #print(elem.text)
                pyear=elem.text
        date_pub=str(pyear)+'-'+str(pmonth)+'-'+str(pday)
        #date_pub=datetime.datetime.strptime(str_pdate, "%Y-%m-%d")

        for a in root.iter('date'):
            x=str(a.attrib)
            #print(a.find('author'))
            if (x.find('received')>-1):
                for elem in a.iter(tag='day'):
                    rday=elem.text
                for elem in a.iter(tag='month'):
                    rmonth=elem.text
                for elem in a.iter(tag='year'):
                    #print(elem.text)
                    ryear=elem.text
        date_rec=str(ryear)+'-'+str(rmonth)+'-'+str(rday)
        #date_rec=datetime.datetime.strptime(str_rdate, "%Y-%m-%d")
        
        for a in root.iter('body'):
            b=ET.tostring(a, method='text', encoding='unicode')
            body=b
        
        for elem in tree.iter(tag='journal-title'):
            journal=elem.text
            
        for elem in tree.iter(tag='journal-id'):
            a=str(elem.attrib)
            if (a.find('publisher-id')>-1):
                publisher_id=elem.text
             
        for elem in tree.iter(tag='journal-meta'):    
            for elem1 in elem.iter(tag='publisher-name'):    
                publisher_name=elem1.text

        mainDF.loc[len(mainDF)]=[doi,pmid,journal,publisher_id,publisher_name,authors,editors,subjects,title,alt_title,date_pub,date_rec,abstract,body]
    return(mainDF)


In [78]:
import time

start = time.time()
DF1=articleToDF(path2)
end = time.time()
time1=(end-start)
DF1.head()

,doi,pmid,journal,publisher_id,publisher_name,authors,editors,subjects,title,alt_title,date_pub,date_rec,abstract,body
0,10.1371/journal.ppat.1003942,24586156,PLoS Pathogens,plos,Public Library of Science,"[John D. Chan, Prince N. Agbedanu, Mostafa Zam...",[Robert M. Greenberg],"[Research Article, Biology, Biochemistry, Drug...",‘Death and Axes’: Unexpected Ca,Ca,2014-2-20,2013-9-20,Author SummarySchistosomiasis (Bilharzia) is o...,IntroductionOver a third of the world's popula...
1,10.1371/journal.ppat.1004337,25166308,PLoS Pathogens,plos,Public Library of Science,"[Marjon Navis, Karen Tran, Shridhar Bale, Gane...",[Ronald C. Desrosiers],"[Research Article, Biology and Life Sciences, ...",HIV-1 Receptor Binding Site-Directed Antibodie...,Vaccine-Elicited VH1-2 HIV Abs in Macaques,2014-8-28,2014-2-25,Author SummaryThe development of an HIV-1 vacc...,IntroductionThe neutralization resistant prope...
2,10.1371/journal.ppat.1005729,27348812,PLoS Pathogens,plos,Public Library of Science,"[Tiffany A. Russell, David C. Tscharke]",[David A. Leib],"[Research Article, Biology and Life Sciences, ...",Lytic Promoters Express Protein during Herpes ...,Lytic Gene Expression in HSV Latency,2016-6-27,2016-3-21,"Author SummaryHerpes simplex virus, which caus...",IntroductionThe herpesviruses are a large fami...
3,10.1371/journal.ppat.1005413,26840124,PLoS Pathogens,plos,Public Library of Science,"[Wei-Yun Wholey, Travis J. Kochan, David N. St...",[Andreas Peschel],"[Research Article, Biology and Life Sciences, ...",Expression of Streptococcus pneumoniae bacteri...,Coordinated Bacteriocin and Competence Express...,2016-2-3,2015-8-25,"Author SummaryThe bacterial pathogen, Streptoc...",IntroductionStreptococcus pneumoniae (pneumoco...
4,10.1371/journal.ppat.1001165,21060817,PLoS Pathogens,plos,Public Library of Science,"[Tracey L. Campbell, Erandi K. De Silva, Kelle...",[Joe D. Smith],"[Research Article, Genetics and Genomics/Gene ...",Identification and Genome-Wide Prediction of D...,ApiAP2 Binding Specificities in,2010-10-28,2010-5-5,Author Summary\nPlasmodium falciparum is the m...,Introduction\nPlasmodium falciparum is respons...


In [43]:
print(time1)

29.56559729576111


In [44]:
DF1.dtypes

doi               object
pmid              object
journal           object
publisher_id      object
publisher_name    object
authors           object
editors           object
subjects          object
title             object
alt_title         object
date_pub          object
date_rec          object
abstract          object
body              object
dtype: object

In [79]:
DF1=DF1.astype('str')

In [14]:
DF1.dtypes

doi               object
pmid              object
journal           object
publisher_id      object
publisher_name    object
authors           object
editors           object
subjects          object
title             object
alt_title         object
date_pub          object
date_rec          object
abstract          object
body              object
dtype: object

6399

In [81]:
DF1.to_sql("/plos_data/plos_pathog", conn, if_exists="replace")

In [7]:
DF2=pd.read_sql("select * from plos_pathog",conn)

DatabaseError: Execution failed on sql 'select * from plos_pathog': no such table: plos_pathog